In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count, countDistinct

import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import min, max

import matplotlib.pyplot as plt
import pandas as pd

from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType
import os

In [2]:
spark = SparkSession.builder \
    .appName("EDA on Mips Overall Performance data") \
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/30 12:36:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [23]:
spark.conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC -XX:InitiatingHeapOccupancyPercent=35")

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/raid/sgupta54/doctor-right/.venv/lib/python3.12/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/raid/sgupta54/doctor-right/.venv/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/raid/sgupta54/doctor-right/.venv/lib/python3.12/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: An error occurred while calling o24.conf

In [3]:
root_path = 'sample-data/doctorright-datalake'

dfs = {}

for i, file in enumerate(os.listdir(root_path)):
    if file.endswith(".parquet"):
        dfs[f"df_{i}"] = spark.read.parquet(f"{root_path}/{file}")

print(dfs.keys())



dict_keys(['df_0', 'df_1', 'df_2', 'df_3', 'df_4', 'df_5', 'df_6', 'df_7'])


In [4]:
for key, df in dfs.items():
    print(key)
    print(df.printSchema())
    

df_0
root
 |-- NPI: integer (nullable = true)
 |-- Ind_PAC_ID: long (nullable = true)
 |-- Ind_enrl_ID: string (nullable = true)
 |-- Provider_Last_Name: string (nullable = true)
 |-- Provider_First_Name: string (nullable = true)
 |-- Provider_Middle_Name: string (nullable = true)
 |-- suff: string (nullable = true)
 |-- gndr: string (nullable = true)
 |-- Cred: string (nullable = true)
 |-- Med_sch: string (nullable = true)
 |-- Grd_yr: integer (nullable = true)
 |-- pri_spec: string (nullable = true)
 |-- sec_spec_1: string (nullable = true)
 |-- sec_spec_2: string (nullable = true)
 |-- sec_spec_3: string (nullable = true)
 |-- sec_spec_4: string (nullable = true)
 |-- sec_spec_all: string (nullable = true)
 |-- Telehlth: string (nullable = true)
 |-- Facility_Name: string (nullable = true)
 |-- org_pac_id: long (nullable = true)
 |-- num_org_mem: integer (nullable = true)
 |-- adr_ln_1: string (nullable = true)
 |-- adr_ln_2: string (nullable = true)
 |-- ln_2_sprs: string (nullabl

In [14]:
df_final = dfs['df_0'].join(dfs['df_1'], ['NPI', 'Ind_PAC_ID'], how='outer')
df_final.printSchema()

root
 |-- NPI: long (nullable = true)
 |-- Ind_PAC_ID: long (nullable = true)
 |-- Ind_enrl_ID: string (nullable = true)
 |-- Provider_Last_Name: string (nullable = true)
 |-- Provider_First_Name: string (nullable = true)
 |-- Provider_Middle_Name: string (nullable = true)
 |-- suff: string (nullable = true)
 |-- gndr: string (nullable = true)
 |-- Cred: string (nullable = true)
 |-- Med_sch: string (nullable = true)
 |-- Grd_yr: integer (nullable = true)
 |-- pri_spec: string (nullable = true)
 |-- sec_spec_1: string (nullable = true)
 |-- sec_spec_2: string (nullable = true)
 |-- sec_spec_3: string (nullable = true)
 |-- sec_spec_4: string (nullable = true)
 |-- sec_spec_all: string (nullable = true)
 |-- Telehlth: string (nullable = true)
 |-- Facility_Name: string (nullable = true)
 |-- org_pac_id: long (nullable = true)
 |-- num_org_mem: integer (nullable = true)
 |-- adr_ln_1: string (nullable = true)
 |-- adr_ln_2: string (nullable = true)
 |-- ln_2_sprs: string (nullable = true

In [4]:
df_0_to_1 = dfs['df_0'].join(dfs['df_1'], ['NPI', 'Ind_PAC_ID'], how='inner')
df_0_to_3 = df_0_to_1.join(dfs['df_3'], ['NPI', 'Ind_PAC_ID'], how='inner')
df_0_to_7 = df_0_to_3.join(dfs['df_7'], ['NPI', 'Ind_PAC_ID'], how='inner')

df_2_to_4 = dfs['df_2'].join(dfs['df_4'], ['Facility_Name', 'org_PAC_ID'], how='inner')
df_2_to_6 = df_2_to_4.join(dfs['df_6'], ['Facility_Name', 'org_PAC_ID'], how='inner')

df_5 = dfs['df_5']

df_final = df_0_to_7.join(df_2_to_6, ['Facility_Name', 'org_PAC_ID'], how='inner')

# Finally, join df_5 using 'virtual_group_id' if needed
# df_final = df_final.join(df_5, ['virtual_group_id'], how='outer')

# Show the final joined dataframe
df_final.show()


24/09/30 12:37:17 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------+----------+---+----------+-----------+------------------+-------------------+--------------------+----+----+----+-------+------+--------+----------+----------+----------+----------+------------+--------+-----------+--------+--------+---------+---------+-----+--------+----------------+---------+---------+-------+---------+------------------+-------------------+--------------------+----+-------------+------------------------------------------+----------------------------------+------------------+-------------------+----------+----------+----------+----------+----------+-------------+--------+-----------------+--------+-------------+----------+-------------------+---------------+--------+----------------+------------------+-------------------+--------------------+----+------------------+-----+----------+-------------------------+----------+-------------+--------+-------------+---+--------+--------+--------+--------+----------+-------------+--------+-----------------+-------

24/09/30 12:37:22 WARN BlockManager: Failed to fetch remote block taskresult_32 from [BlockManagerId(driver, cadejo.cs.usfca.edu, 41843, None)] after 1 fetch failures. Most recent failure cause:
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.network.BlockTransferService.fetchBlockSync(BlockTransferService.scala:103)
	at org.apache.spark.storage.BlockManager.fetchRemoteManagedBuffer(BlockManager.scala:1172)
	at org.apache.spark.storage.BlockManager.$anonfun$getRemoteBlock$8(BlockManager.scala:1116)
	at scala.Option.orElse(Option.scala:447)
	at org.apache.spark.storage.BlockManager.getRemoteBlock(BlockManager.scala:1116)
	at org.apache.spark.storage.BlockManager.getRemoteBytes(BlockManager.scala:1256)
	at org.apache.spark.scheduler.TaskResultGetter$$anon$3.$anonfun$run$1(TaskResultGetter.scala